In [1]:
!nvidia-smi

Sat Aug 14 21:35:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 466.81       Driver Version: 466.81       CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   36C    P8     6W /  N/A |    105MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import tensorflow as tf

tf.device('/CPU:0'), tf.device('/GPU:0'), tf.device('/GPU:1')

(<tensorflow.python.eager.context._EagerDeviceContext at 0x2ec58bf8100>,
 <tensorflow.python.eager.context._EagerDeviceContext at 0x2ec58bf80c0>)

In [3]:
len(tf.config.experimental.list_physical_devices('GPU'))

1

In [4]:
def try_gpu(i=0):
    if len(tf.config.experimental.list_physical_devices('GPU')) >= i + 1:
        return tf.device(f'/GPU:{i}')
    return tf.device('/CPU:0')

def try_all_gpus():
    num_gpus = len(tf.config.experimental.list_physical_devices('GPU'))
    devices = [tf.device(f'/GPU:{i}') for i in range(num_gpus)]
    return devices if devices else [tf.device('/CPU:0')]

try_gpu(), try_gpu(10), try_all_gpus()

(<tensorflow.python.eager.context._EagerDeviceContext at 0x2ec5c0bed80>,
 [<tensorflow.python.eager.context._EagerDeviceContext at 0x2ec02100100>])

In [5]:
x = tf.constant([1, 2, 3])
x.device

'/job:localhost/replica:0/task:0/device:CPU:0'

In [6]:
with try_gpu():
    x = tf.ones((2, 3))
x

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)>

In [7]:
with try_gpu(1):
    y = tf.random.uniform((2, 3))
y

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.58698475, 0.9504894 , 0.7498311 ],
       [0.45935214, 0.05873632, 0.7739111 ]], dtype=float32)>

In [8]:
with try_gpu(1):
    z = x
print(x)
print(z)

tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)


In [9]:
y + z

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1.5869848, 1.9504894, 1.7498311],
       [1.4593521, 1.0587363, 1.7739111]], dtype=float32)>

In [10]:
with try_gpu(1):
    z2 = z
z2 is z

True

In [11]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    net = tf.keras.models.Sequential([tf.keras.layers.Dense(1)])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [12]:
net(x)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[1.2256849],
       [1.2256849]], dtype=float32)>

In [13]:
net.layers[0].weights[0].device, net.layers[0].weights[1].device

('/job:localhost/replica:0/task:0/device:GPU:0',
 '/job:localhost/replica:0/task:0/device:GPU:0')